A shout out to H-Z-Ning and his notebook
<a href="https://www.kaggle.com/code/hzning/top-10-solution-0-97525-esay-is-all-you#%F0%9F%8F%81-LightGBM-Training">🔥 Top 10% Solution|🏆0.97525,esay is all you!🚀</a>
for feature engineering ideas

## Load libraries

In [1]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import TargetEncoder, FunctionTransformer, Binarizer, OrdinalEncoder
import imblearn.pipeline as imb
from sklearn.model_selection import train_test_split
from pathlib import Path
import os
from sklearn import set_config
from imblearn.over_sampling import SMOTE
import sys
import xgboost as xgb

from utils import convert_months_to_categorical, calculate_score, ClipValues, SinePreprocess, CategoryCounter

# Enable pandas output globally
set_config(transform_output="pandas")

## Load data

In [2]:

train_file_path = "train.csv"
test_file_path = "test.csv"

In [3]:
original_train_df = pd.read_csv(train_file_path, index_col=0)
test_df = pd.read_csv(test_file_path, index_col=0)

In [4]:
y_var = "y"
categorical_variables = original_train_df.select_dtypes(include="object").columns.to_list()
numerical_variables = (
    original_train_df
    .select_dtypes(exclude="object")
    .columns
    .drop(
        labels=["y"]
    )
    .to_list()
)

In [5]:
original_train_df.tail()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
id,,,,,,,,,,,,,,,,,
749995,29,services,single,secondary,no,1282,no,yes,unknown,4,jul,1006,2,-1,0,unknown,1
749996,69,retired,divorced,tertiary,no,631,no,no,cellular,19,aug,87,1,-1,0,unknown,0
749997,50,blue-collar,married,secondary,no,217,yes,no,cellular,17,apr,113,1,-1,0,unknown,0
749998,32,technician,married,secondary,no,-274,no,no,cellular,26,aug,108,6,-1,0,unknown,0
749999,42,technician,married,secondary,no,1559,no,no,cellular,4,aug,143,1,1,7,failure,0


## Train val split

In [6]:
train_df, val_df = train_test_split(original_train_df, test_size=0.2)

## Preprocessing and train Pipeline

In [7]:
import numpy as np

In [23]:
def force_category(df):
    # cat_cols = ["default", "housing", "loan", "poutcome", "education", "contact", "job", "marital"]
    for col in df.select_dtypes(include="object").columns:
        df[col] = pd.Categorical(df[col])
    return df

def add_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    # df["balance_negative_times_default_yes"] = (df["balance"] < 0) * (df['default'] == "yes")
    # df["balance_negative_times_loan_yes"] = (df["balance"] < 0) * (df['loan'] == "yes")
    # df["balance_negative"] = (df["balance"] < 0).astype(int)
    # df["balance_log"] = np.log1p(df['balance'].clip(lower=0))
    # df["default_yes_times_loan_yes"] = (df['default'] == "yes") * (df['loan'] == "yes")
    # df["contact_unknown_times_default_yes"] = (df['contact'] == "unknown") * (df['default'] == "yes")
    # df["duration_times_loan_yes"] = (df["duration"]) * (df['loan'] == "yes")
    # df["duration_times_housing_yes"] = (df["duration"]) * (df['housing'] == "yes")
    # df["duration_times_contact_unknown"] = (df["duration"]) * (df['contact'] == "unknown")
    # df["duration_times_age"] = (df["duration"] < 53) * (df["age"])
    # df["balance_negative_times_poutcome_unknown"] = (df["balance"] < 0) * (df['poutcome'] == "unknown")
    # df["duration_times_default_yes"] = (df["duration"]) * (df['default'] == "yes")
    # df["duration_times_education_primary"] = (df["duration"]) * (df['education'] == "primary")
    df["pdays_by_campaign"] = (df["pdays"]) / (df['campaign'])
    df["previous_by_campaign"] = (df["previous"]) / (df['campaign'] + 1e-8)
    df["pdays_times_previous"] = (df["pdays"]) * (df['previous'])
    df["day_of_year"] = df["month"].cat.codes * 30 + df["day"]
    # df["duration_sq"] = df["duration"] ** 2
    df['job_edu'] = df['job'].astype(str) + "_" + df['education'].astype(str)
    return df


power_pipeline = Pipeline(
    [
        ("passthrough", "passthrough"),
        # ("clip", ClipValues(quantile_range=(0.25, 0.75))),
        # ("power_transform", PowerTransformer("yeo-johnson", standardize=True)),
        # ("splines", SplineTransformer(n_knots=10, knots="quantile"))
    ],
)

cat_encoder = Pipeline(
    [
        ("encoder", CategoryCounter()),
        
    ]
)

target_encoder = TargetEncoder()
sin_preprocess = SinePreprocess(
    {
        "day": 31,
        "month": 12,
        "duration": 400,
        "day_of_year": 360
    }
)

preprocessing_pipeline_0 = ColumnTransformer(
    [
        ("numerical", power_pipeline, [
            "balance", "duration", "campaign", "pdays", "age",
        ]),
        ("binarizer", Binarizer(threshold=0.01), ["balance", "pdays"]),
        ("encode", "passthrough", ["default", "housing", "loan", "poutcome", "education", "contact", "job", "marital"]),
        ("target_encoder", target_encoder, ["day", "month", "default", "housing", "loan", "poutcome", "education", "contact", "job", "marital", "job_edu"]),
        ("sine_preproces", sin_preprocess, ["day", "month", "duration", "day_of_year"]),
        ("drop", "drop", ["previous"])
    ],
    remainder="passthrough"
    # verbose_feature_names_out=False
)

def create_pipeline(params = {}):

    pipeline = Pipeline(
        [
            ("force_cats", FunctionTransformer(force_category, validate=False)),
            ("month_transform_to_categorical", FunctionTransformer(
                convert_months_to_categorical, 
                validate=False,
            )),
            ("add_columns", FunctionTransformer(
                add_columns, 
                validate=False,
            )),
            ("preprocess_0", preprocessing_pipeline_0),
            ("drop_not_needed", 
            ColumnTransformer(transformers=[
                ("drop", "drop", []),
            ], remainder="passthrough", verbose_feature_names_out=False)
            ),
            # ("smote", SMOTE(random_state=42)),
            ("classifier", xgb.XGBClassifier(
                tree_method="hist", 
                enable_categorical=True,
                objective="binary:logistic",
                eval_metric="logloss",
                device="cuda",
                **params
            ))
        ]
    )
    return pipeline

In [43]:
pipeline = create_pipeline({
    "n_estimators": 50000,
    "eta": 0.02,
    "colsample_bytree": 0.6,
    "reg_alpha": 3.7,
    "reg_lambda": 4.7,
    "max_depth": 6,
    "subsample": 0.8,
    'max_leaves': 225,
    "gamma": 1,
    "scale_pos_weight": 0.5,
})
pipeline.fit(
    train_df.drop("y", axis=1), 
    train_df[y_var], 
)

,steps,"[('force_cats', ...), ('month_transform_to_categorical', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,func,<function for...x7f01adbbc1f0>
,inverse_func,None
,validate,False
,accept_sparse,False
,check_inverse,True
,feature_names_out,None
,kw_args,None


## Evaluate

In [46]:
train_pred_probs = pipeline.predict_proba(train_df)[:, 1]
calculate_score(train_df["y"], train_pred_probs)

{'auc': 0.981148205793965, 'accuracy': 0.94911, 'f1': 0.7539366588766218}

In [47]:
val_pred_probs = pipeline.predict_proba(val_df)[:, 1]
calculate_score(val_df["y"], val_pred_probs)

{'auc': 0.9711235066539389,
 'accuracy': 0.9367466666666666,
 'f1': 0.6935004522548133}

## Review

In [48]:
pipeline[-1].get_booster().get_score(importance_type='gain')

{'numerical__balance': 3.684072256088257,
 'numerical__duration': 66.53909301757812,
 'numerical__campaign': 3.8592946529388428,
 'numerical__pdays': 6.4564948081970215,
 'numerical__age': 3.6198344230651855,
 'binarizer__balance': 4.808575630187988,
 'binarizer__pdays': 7.170206069946289,
 'encode__default': 2.469078302383423,
 'encode__housing': 53.032142639160156,
 'encode__loan': 14.588165283203125,
 'encode__poutcome': 52.77151870727539,
 'encode__education': 2.933241128921509,
 'encode__contact': 39.44669723510742,
 'encode__job': 3.310016632080078,
 'encode__marital': 3.793721914291382,
 'target_encoder__day': 3.7709012031555176,
 'target_encoder__month': 15.768763542175293,
 'target_encoder__default': 1.8841915130615234,
 'target_encoder__housing': 8.33703327178955,
 'target_encoder__loan': 3.6738197803497314,
 'target_encoder__poutcome': 17.66435432434082,
 'target_encoder__education': 1.9384336471557617,
 'target_encoder__contact': 7.768923282623291,
 'target_encoder__job': 2

## Submission

In [49]:
test_pred_probs = pipeline.predict_proba(test_df)
test_pred_probs

array([[9.9879879e-01, 1.2012240e-03],
       [9.5658863e-01, 4.3411396e-02],
       [9.9988949e-01, 1.1049058e-04],
       ...,
       [5.5193633e-01, 4.4806367e-01],
       [9.9958336e-01, 4.1664598e-04],
       [9.0373117e-01, 9.6268840e-02]], shape=(250000, 2), dtype=float32)

In [50]:
submission = test_df[[]]
submission = submission.copy()
submission.loc[:, "y"] = test_pred_probs[:, 1]
submission.to_csv("submission_xgb.csv")